![Alt text](https://cdn1.img.sputniknews.com/img/105872/94/1058729488_0:339:3161:2046_1000x541_80_0_0_071f83c13cdfdf01475883e69b1089f9.jpg "Covid-19")


# Explore a dataset on the modern Olympic Games, including all the Games from Athens 1896 to Rio 2016

- comments: true
- badges: true
- catagories: [Python, Pandas, numpy]
- image: images/Olympic.png

##### The Olympic Games, which originated in ancient Greece as many as 3,000 years ago, were revived in the late 19th century and have become the world’s preeminent sporting competition. From the 8th century B.C. to the 4th century A.D., the Games were held every four years in Olympia, located in the western Peloponnese peninsula, in honor of the god Zeus. The first modern Olympics took place in 1896 in Athens, and featured 280 participants from 13 nations, competing in 43 events. Since 1994, the Summer and Winter Olympic Games have been held separately and have alternated every two years.

##### **In this Project we have to analysis the data of olympic game by using python programing.**

##### **Let’s Start with importing the libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

### Collecting information about both the data sets

### We are going to:
1. Review the first lines of the data;
2. Use the describe and info functions to collect statistical information, datatypes, column names and other information

In [ ]:
data = pd.read_csv('../input/athlete-events/athlete_events.csv')
regions = pd.read_csv('../input/athlete-events/noc_regions.csv')

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
regions = pd.read_csv('../input/athlete-events/noc_regions.csv')
regions.head()

## Joining the data frames

In [ ]:
merged = pd.merge(data, regions, on='NOC', how='left')
merged.head()

## Distribution of the age of gold medalists
### Let’s start creating a new data frame including only gold medalists.

In [ ]:
goldMedals = merged[(merged.Medal == 'Gold')]
goldMedals.head()

### I would like to have a plot of the Age to see the distribution but I need to check first if the Age column contains NaN values.

In [ ]:
goldMedals.isnull().any()

### Let’s take only the values that are different from NaN.

In [ ]:
goldMedals = goldMedals[np.isfinite(goldMedals['Age'])]

### We can now create a countplot to see the result of our work:

In [ ]:
plt.figure(figsize=(20, 10))
plt.tight_layout()
sns.countplot(goldMedals['Age'])
plt.title('Distribution of Gold Medals')

### It seems that we have people with Age greater that 50 with a gold medal: Let’s know more about those people.

In [ ]:
goldMedals['ID'][goldMedals['Age'] > 50].count()

### 65 people. Wonderul But which disciplines allows you to land a gold medal after your fifties?

### We will now create a new dataframe called masterDisciplines in which we will insert this new set of people and then create a visualization with it.

In [ ]:
masterDisciplines = goldMedals['Sport'][goldMedals['Age'] > 50]
plt.figure(figsize=(20, 10))
plt.tight_layout()
sns.countplot(masterDisciplines)
plt.title('Gold Medals for Athletes Over 50')

### It seems that our senior gold medalists are shooters, archers, sailors and, above all, horse riders!

### It makes sense: I cannot imagine a sprinter making 100 meters in 10 seconds at 55, but who knows!

## Women in Athletics
### Studying the data we can try to understand how much medals we have only for women in the recent history of the Summer Games. Let’s create a filtered dataset :

In [ ]:
womenInOlympics = merged[(merged.Sex == 'F') & (merged.Season == 'Summer')]
womenInOlympics.head(10)

### To plot the curve over time, let’s create a plot in which we put the year (on the x-axis) and count of the number of medals per edition of the games (consider that we will have more medals for the same athlete).

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize=(20, 10))
sns.countplot(x='Year', data=womenInOlympics)
plt.title('Women medals per edition of the Games')

### Usually I cross-check the data: below I tried to review only the medalists for the 1900 Summer edition to see if the visualization is correct.

In [ ]:
womenInOlympics.loc[womenInOlympics['Year'] == 1900].head(10)

### Okay, let’s count the rows (same code as above adding the count() function and filtering only for ID)

In [ ]:
womenInOlympics['ID'].loc[womenInOlympics['Year'] == 1900].count()

### So we have 33 records (with repetitions, for example ‘Marion Jones (-Farquhar)’ won a medal both for Tennis Women’s Singles and Tennis Mixed Doubles – To be sure I cross-checked also with Wikipedia and the outcome seems correct).

## Medals per country
### Let’s now review the top 5 gold medal countries:

In [ ]:
goldMedals.region.value_counts().reset_index(name='Medal').head()

### Let’s plot this:

In [ ]:
totalGoldMedals = goldMedals.region.value_counts().reset_index(name='Medal').head(5)
g = sns.catplot(x="index", y="Medal", data=totalGoldMedals,
                height=6, kind="bar", palette="muted")
g.despine(left=True)
g.set_xlabels("Top 5 countries")
g.set_ylabels("Number of Medals")
plt.title('Medals per Country')

### The USA seems to be the most winning country.
### But which are the most awarded disciplines of American Athletes?


 
# Disciplines with the greatest number of Gold Medals
### Let’s create a dataframe to filter the gold medals only for the USA.

In [ ]:
goldMedalsUSA = goldMedals.loc[goldMedals['NOC'] == 'USA']

### Done! Now, we can count the medals per discipline

In [ ]:
goldMedalsUSA.Event.value_counts().reset_index(name='Medal').head(20)

### Let’s slice the dataframe using only the data of male athletes to better review it:

In [ ]:
basketballGoldUSA = goldMedalsUSA.loc[(goldMedalsUSA['Sport'] == 'Basketball') & (goldMedalsUSA['Sex'] == 'M')].sort_values(['Year'])
basketballGoldUSA.head(15)

### What we supposed is true: the medals are not grouped by Edition/Team but we were counting the gold medals of each member of the team!


### Let’s proceed grouping by year the athletes – the idea is to create a new dataframe to make a pre-filter using only the first record for each member of the team.

In [ ]:
groupedBasketUSA = basketballGoldUSA.groupby(['Year']).first()
groupedBasketUSA
groupedBasketUSA['ID'].count()

## What is the median height/weight of an Olympic medalist?

### Let’s try to plot a scatterplot of height vs weight to see the distribution of values (without grouping by discipline).
### First of all, we have to take again the goldMedals dataframe

In [ ]:
goldMedals.head()

### We can see that we have NaN values both in height and weight columns
### At this point, we can act as follows:

1. Using only the rows that has a value in the Height and Weight columns;
1. Replace the value with the mean of the column.

### Solution 2 in my opinion it is not the best way to go: we are talking about data of athletes of different ages and different disciplines (that have done different training).

### Let’s go with solution 1.

### The first thing to do is to collect general information about the dataframe that we have to use: goldMedals.

In [ ]:
goldMedals.info()

### Okay, we have more than 13.000 rows.

### We will now create a dataframe filtering only the rows that has the column Height and Weight populated.

In [ ]:
notNullMedals = goldMedals[(goldMedals['Height'].notnull()) & (goldMedals['Weight'].notnull())]

In [ ]:
plt.figure(figsize=(12, 10))
ax = sns.scatterplot(x="Height", y="Weight", data=notNullMedals)
plt.title('Height vs Weight of Olympic Medalists')

### The vast majority of the samples show a linear relation between height and weight (the more the weight, the more the height).

### We have exceptions and I am willing to know more!

### For example, let’s see which is the athlete that weighs more than 160 kilograms.

In [ ]:
notNullMedals.loc[notNullMedals['Weight'] > 160]

In [ ]:
MenOverTime = merged[(merged.Sex == 'M') & (merged.Season == 'Summer')]
WomenOverTime = merged[(merged.Sex == 'F') & (merged.Season == 'Summer')]

In [ ]:
MenOverTime.head()

In [ ]:
part = MenOverTime.groupby('Year')['Sex'].value_counts()
plt.figure(figsize=(20, 10))
part.loc[:,'M'].plot()
plt.title('Variation of Male Athletes over time')

In [ ]:
part = WomenOverTime.groupby('Year')['Sex'].value_counts()
plt.figure(figsize=(20, 10))
part.loc[:,'F'].plot()
plt.title('Variation of Female Athletes over time')

### What I immediately saw is that for women:

1. We have a steep increase in the population;
1. The grow is constant.

### On the other hand, the grow for men seems less strong:
1. After the 1990 we can see a relevant decrease in the number of male athletes at the summer games;
1. The growth has slowly restarted recently.

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot('Year', 'Age', data=MenOverTime)
plt.title('Variation of Age for Male Athletes over time')

### What is strange for me is the age of some athletes in the games between the 1924 and the 1948: let’s check all the people with age greater than 80.

In [ ]:
MenOverTime.loc[MenOverTime['Age'] > 80].head(10)

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot('Year', 'Age', data=WomenOverTime)
plt.title('Variation of Age for Female Athletes over time')

### Interesting points for me:

* Generally, the age distribution starts has a lower minimum and a lower maximum;
* In 1904 the age distribution is strongly different from the other Olympics: let’s know more about this point:

In [ ]:
WomenOverTime.loc[WomenOverTime['Year'] == 1904]

### We will now try using a pointplot to visualize the variation in weight over athletes.
### The first graph will show data for men, the second for women:

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Weight', data=MenOverTime)
plt.title('Variation of Weight for Male Athletes over time')

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Weight', data=WomenOverTime)
plt.title('Variation of Weight for Female Athletes over time')

### What we can see is that it seems that we do not have data for women before 1924. Let’s try filtering all the women athletes for that period to review this point:

In [ ]:
womenInOlympics.loc[womenInOlympics['Year'] < 1924].head(20)

### Using the same pointplot (with a different palette) we can plot the weight change along time.

### The first graph will show the information for men, the second for women:

In [ ]:
womenInOlympics.loc[womenInOlympics['Year'] < 1924].head(20)

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Height', data=WomenOverTime, palette='Set2')
plt.title('Variation of Height for Female Athletes over time')

### What we may see:

* For both men and women, the height is incrementing over time but it is decreasing between the 2012 and the 2016.
* For women we have a peak between 1928 and 1948, let’s deepen this point:

In [ ]:
WomenOverTime.loc[(WomenOverTime['Year'] > 1924) & (WomenOverTime['Year'] < 1952)].head(10)

In [ ]:
MenOverTime.head()

In [ ]:
itMenOverTime = MenOverTime.loc[MenOverTime['region'] == 'Italy']

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize=(20, 10))
sns.countplot(x='Year', data=itMenOverTime, palette='Set2')
plt.title('Variation of Age for Italian Male Athletes over time')

### now we can plot the change over time:

In [ ]:
itWomenOverTime = WomenOverTime.loc[WomenOverTime['region'] == 'Italy']
sns.set(style="darkgrid")
plt.figure(figsize=(20, 10))
sns.countplot(x='Year', data=itWomenOverTime, palette='Set2')
plt.title('Variation of Age for Italian Female Athletes over time')

### Let’s first of all isolate all the discipline of the Olympics dataframe.

### My idea is to see if Gymnastics is called differently or if there is any type.

In [ ]:
MenOverTime['Sport'].unique().tolist()

### Okay, the string to use to filter is ‘Gymnastics’: let’s create two new dataframes for men and women.

In [ ]:
gymMenOverTime = MenOverTime.loc[MenOverTime['Sport'] == 'Gymnastics']
gymWomenOverTime = WomenOverTime.loc[WomenOverTime['Sport'] == 'Gymnastics']

### Okay: let’s now create our plot for male and female athletes and then we can make our observations

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot('Year', 'Weight', data=gymMenOverTime)
plt.title('Weight over year for Male Gymnasts')

### A few things I noticed:

1. The weight for female Gymnasts has go down for 60 to 50 kilograms on average;
1. The weight for men has been more or less stable since 1964;
1. The height is more stable for both men and women.
### Also, men weight data from 1924 seems missing: let’s check.

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot('Year', 'Height', data=gymMenOverTime)
plt.title('Height over year for Male Gymnasts')

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot('Year', 'Weight', data=gymWomenOverTime)
plt.title('Weight over year for Female Gymnasts')

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot('Year', 'Height', data=gymWomenOverTime)
plt.title('Height over year for Female Gymnasts')

In [ ]:
gymMenOverTime['Weight'].loc[gymMenOverTime['Year'] == 1924].isnull().all()

## Weightlifting
### Let’s work on an analysis similar to what we have done for Gymnastics also for the Lifters.
### We can start creating a new, dedicated dataframe

In [ ]:
wlMenOverTime = MenOverTime.loc[MenOverTime['Sport'] == 'Weightlifting']
wlWomenOverTime = WomenOverTime.loc[WomenOverTime['Sport'] == 'Weightlifting']

### Okay: let’s now create our plot for male and female athletes and then we can make our observations

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Weight', data=wlMenOverTime, palette='Set2')
plt.title('Weight over year for Male Lifters')

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Height', data=wlMenOverTime, palette='Set2')
plt.title('Height over year for Male Lifters')

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Weight', data=wlWomenOverTime)
plt.title('Weight over year for Female Lifters')

In [ ]:
plt.figure(figsize=(20, 10))
sns.pointplot('Year', 'Height', data=wlWomenOverTime)
plt.title('Height over year for Female Lifters')